In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import dotenv
import os
import json
import requests 
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')

In [2]:
ct = contrans()

In [3]:
goodbioguide = ct.get_bioguide(name='good', state='Virginia', district=5)
goodbioguide

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,G000595,5.0,"good, bob",Republican,Virginia,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000595?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000595_20...,"[{'chamber': 'House of Representatives', 'star..."
1,G000280,5.0,"goode, virgil h., jr.",Republican,Virginia,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/G000280?for...,"Image, Congressional Pictorial Directory, 109th.",https://www.congress.gov/img/member/g000280_20...,"[{'chamber': 'House of Representatives', 'endY..."


In [4]:
goodid  = goodbioguide.reset_index()['bioguideId'][0]

In [5]:
goodsbills = ct.get_sponsoredlegislation(goodid)

In [6]:
billurl = goodsbills[0]['url'] ### this is an example of one bill

In [7]:
ct.get_billdata(billurl)

{'bill': {'actions': {'count': 3,
   'url': 'https://api.congress.gov/v3/bill/118/hres/1509/actions?format=json'},
  'committees': {'count': 1,
   'url': 'https://api.congress.gov/v3/bill/118/hres/1509/committees?format=json'},
  'congress': 118,
  'cosponsors': {'count': 30,
   'countIncludingWithdrawnCosponsors': 30,
   'url': 'https://api.congress.gov/v3/bill/118/hres/1509/cosponsors?format=json'},
  'introducedDate': '2024-09-25',
  'latestAction': {'actionDate': '2024-09-25',
   'text': 'Referred to the House Committee on Energy and Commerce.'},
  'number': '1509',
  'originChamber': 'House',
  'originChamberCode': 'H',
  'sponsors': [{'bioguideId': 'G000595',
    'district': 5,
    'firstName': 'Bob',
    'fullName': 'Rep. Good, Bob [R-VA-5]',
    'isByRequest': 'N',
    'lastName': 'Good',
    'party': 'R',
    'state': 'VA',
    'url': 'https://api.congress.gov/v3/member/G000595?format=json'}],
  'textVersions': {'count': 1,
   'url': 'https://api.congress.gov/v3/bill/118/hres/